# MGIMO intensive

## NTL dataset: CNN approach

### 1. Libraries

Documentation for use of OpenCV with Python API [see here](https://docs.opencv.org/).

In [ ]:
!pip3 install opencv-python

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from ydata_profiling import ProfileReport

import re
from typing import Tuple, Dict, List, Optional, Union

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupShuffleSplit
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from PIL import Image, ImageFile
import gc

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Включение поддержки загрузки больших изображений
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
IMG_PATH = '/home/jovyan/__DATA/mgimo_intensive/nlt/dataset'
TARGET_SIZE = (128, 128)
LIMIT_IMGS = 2000
RANDOM_STATE = 42
BATCH_SIZE = 16
LEARNING_RATE = 0.001
NUM_EPOCHS = 20

### 2. Explore the data

Take the dataset from Kaggle - [Country-Wise Nightlight Images Dataset](https://www.kaggle.com/datasets/abhijeetdtu/country-nightlight-dataset).

In [ ]:
DATA_PATH = '/home/jovyan/__DATA/mgimo_intensive/nlt'

In [ ]:
!ls -la $DATA_PATH

In [ ]:
data_dir = Path(IMG_PATH)

### 3. Panel data

In [ ]:
df_gdp = pd.read_csv(f"{DATA_PATH}/gdp_melted.csv", index_col=0)

In [ ]:
df_gdp.info()

In [ ]:
df_gdp.head()

### 4. Images data

In [ ]:
def create_image_dataframe(img_path: str) -> pd.DataFrame:
    """Создает датафрейм с информацией об изображениях.
    
    Args:
        img_path: Путь к директории с изображениями
        
    Returns:
        DataFrame с колонками:
        - country_code: код страны из названия файла
        - year: год из названия файла
        - file_path: полный путь к файлу изображения
        - filename: исходное имя файла
        
    Raises:
        FileNotFoundError: Если указанная директория не существует
    """
    
    if not os.path.exists(img_path):
        raise FileNotFoundError(f"Директория {img_path} не найдена")
    
    # Собираем информацию о файлах
    image_data = []
    
    # Паттерн для извлечения кода страны и года из имени файла
    # Пример: CHN_1997.png_0_4784.jpeg
    pattern = re.compile(r'([A-Z]{3})_(\d{4})')
    
    for filename in os.listdir(img_path):
        if not filename.lower().endswith(('.jpeg', '.jpg', '.png')):
            continue
            
        match = pattern.search(filename)
        if match:
            country_code = match.group(1)
            year = int(match.group(2))
            
            image_data.append({
                'country_code': country_code,
                'year': year,
                'file_path': str(Path(img_path) / filename),
                'filename': filename
            })
    
    df_img = pd.DataFrame(image_data)
    
    if df_img.empty:
        print("Предупреждение: не найдено изображений, соответствующих паттерну")
    
    return df_img


def load_and_resize_image(file_path: str, target_size: Tuple[int, int] = (224, 224)) -> np.ndarray:
    """Загружает изображение и изменяет его до точного размера target_size.
    
    Args:
        file_path: Путь к файлу изображения
        target_size: Целевой размер (ширина, высота), по умолчанию (224, 224)
        
    Returns:
        Изображение в виде numpy массива (224, 224) с типом float16
    """
    try:
        # Загружаем изображение сразу в grayscale
        img = cv2.imread(file_path, cv2.IMREAD_REDUCED_GRAYSCALE_2)
        
        if img is None:
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
            
        if img is None:
            print(f"Не удалось загрузить изображение: {file_path}")
            return None
        
        # Принудительный ресайз до точного размера 224x224
        # INTER_LINEAR хорош для увеличения, INTER_AREA для уменьшения
        # Определяем, нужно увеличивать или уменьшать
        height, width = img.shape[:2]
        if height > target_size[1] or width > target_size[0]:
            # Уменьшаем
            interpolation = cv2.INTER_AREA
        else:
            # Увеличиваем
            interpolation = cv2.INTER_LINEAR
        
        # Ресайз до точного размера
        img_resized = cv2.resize(img, target_size, interpolation=interpolation)
        
        # Конвертируем в float16 и нормализуем
        img_array = img_resized / 255.0
        
        # Очищаем память
        del img, img_resized
        gc.collect()
        
        return img_array
        
    except Exception as e:
        print(f"Ошибка при загрузке {file_path}: {e}")
        return None


def extract_image_features(
    image_array: np.ndarray,
    method: str = 'flatten'
) -> np.ndarray:
    """Извлекает признаки из черно-белого изображения.
    
    Args:
        image_array: Изображение в виде numpy массива (height, width)
        method: Метод извлечения признаков
        
    Returns:
        Вектор признаков
    """
    if image_array is None:
        return None
    
    if method == 'flatten':
        # Для grayscale просто выпрямляем 2D массив
        features = image_array.flatten()
    elif method == 'mean_pooling':
        # Усреднение по блокам для уменьшения размерности
        h, w = image_array.shape
        block_size = 8
        h_blocks = h // block_size
        w_blocks = w // block_size
        features = image_array[:h_blocks*block_size, :w_blocks*block_size] \
                          .reshape(h_blocks, block_size, w_blocks, block_size) \
                          .mean(axis=(1, 3)) \
                          .flatten()
    else:
        raise ValueError(f"Неизвестный метод: {method}")
    
    return features


def prepare_features_for_country_year(
    df_img: pd.DataFrame,
    method: str = 'flatten',
    limit: int = 200,
) -> pd.DataFrame:
    """Подготавливает признаки для всех изображений.
    
    Args:
        df_img: Датафрейм с информацией об изображениях
        method: Метод извлечения признаков
        
    Returns:
        DataFrame с признаками и мета-информацией
    """
    
    feature_list = []

    if limit == None: limit = len(df_img)
    
    for idx, row in df_img[:limit].iterrows():
        if idx % 100 == 0:  # Прогресс каждые 100 изображений
            print(f"Обработано {idx} из {len(df_img)} изображений")
        
        # Загружаем и уменьшаем изображение
        img_array = load_and_resize_image(row['file_path'], TARGET_SIZE)
        
        if img_array is not None:
            # Извлекаем признаки
            features = extract_image_features(img_array, method)
            
            # Сохраняем признаки вместе с мета-информацией
            feature_dict = {
                'country_code': row['country_code'],
                'year': row['year'],
                'file_path': row['file_path']
            }
            
            # Добавляем признаки как отдельные колонки
            for i, feature_value in enumerate(features):
                feature_dict[f'feature_{i}'] = feature_value
            
            feature_list.append(feature_dict)
    
    df_features = pd.DataFrame(feature_list)
    
    return df_features


def merge_with_gdp_data(
    df_features: pd.DataFrame,
    df_gdp: pd.DataFrame
) -> pd.DataFrame:
    """Объединяет признаки с данными о ВВП.
    
    Args:
        df_features: DataFrame с признаками изображений
        df_gdp: DataFrame с данными о ВВП
        
    Returns:
        Объединенный DataFrame
    """
    
    # Подготавливаем данные ВВП
    df_gdp_filtered = df_gdp[['code', 'year', 'gdp']].copy()
    df_gdp_filtered = df_gdp_filtered.dropna(subset=['gdp'])
    
    # Переименовываем для объединения
    df_gdp_filtered = df_gdp_filtered.rename(columns={'code': 'country_code'})
    
    # Объединяем датафреймы
    df_merged = pd.merge(
        df_features,
        df_gdp_filtered,
        on=['country_code', 'year'],
        how='inner'
    )
    
    print(f"Объединено записей: {len(df_merged)}")
    print(f"Уникальных стран: {df_merged['country_code'].nunique()}")
    print(f"Диапазон лет: {df_merged['year'].min()} - {df_merged['year'].max()}")
    
    return df_merged


def split_data_by_country_and_year(
    df: pd.DataFrame,
    test_size: float = 0.2,
    val_size: float = 0.1
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """Разделяет данные на train/val/test с учетом стран и временных периодов.
    
    Args:
        df: DataFrame с данными
        test_size: Доля тестовой выборки
        val_size: Доля валидационной выборки
        
    Returns:
        Кортеж (train_df, val_df, test_df)
    """
    
    # Создаем группы по странам
    groups = df['country_code'].values
    
    # Первое разделение: выделяем тестовую выборку
    gss1 = GroupShuffleSplit(
        n_splits=1,
        test_size=test_size,
        random_state=RANDOM_STATE
    )
    
    train_val_idx, test_idx = next(
        gss1.split(df, groups=groups)
    )
    
    train_val_df = df.iloc[train_val_idx]
    test_df = df.iloc[test_idx]
    
    # Второе разделение: выделяем валидационную из train
    val_relative_size = val_size / (1 - test_size)
    gss2 = GroupShuffleSplit(
        n_splits=1,
        test_size=val_relative_size,
        random_state=RANDOM_STATE
    )
    
    train_idx, val_idx = next(
        gss2.split(
            train_val_df,
            groups=train_val_df['country_code'].values
        )
    )
    
    train_df = train_val_df.iloc[train_idx]
    val_df = train_val_df.iloc[val_idx]
    
    print(f"Train: {len(train_df)} samples, {train_df['country_code'].nunique()} countries")
    print(f"Val: {len(val_df)} samples, {val_df['country_code'].nunique()} countries")
    print(f"Test: {len(test_df)} samples, {test_df['country_code'].nunique()} countries")
    
    return train_df, val_df, test_df


def prepare_X_y(
    df: pd.DataFrame,
    feature_cols: List[str],
    target_col: str = 'gdp'
) -> Tuple[np.ndarray, np.ndarray]:
    """Подготавливает матрицу признаков и целевую переменную.
    
    Args:
        df: DataFrame с данными
        feature_cols: Список колонок с признаками
        target_col: Название колонки с целевой переменной
        
    Returns:
        Кортеж (X, y)
    """
    
    X = df[feature_cols].values
    y = df[target_col].values
    
    return X, y

#### 5.1. Data preprocessing

In [ ]:
print("=" * 50)
print("ШАГ 1: Создание датафрейма изображений")
print("=" * 50)

df_img = create_image_dataframe(IMG_PATH)
print(f"Найдено изображений: {len(df_img)}")
print(f"Уникальных стран: {df_img['country_code'].nunique()}")
print(f"Диапазон лет: {df_img['year'].min()} - {df_img['year'].max()}")

In [ ]:
df_img.head()

In [ ]:
print("\n" + "=" * 50)
print("ШАГ 2: Извлечение признаков из изображений")
print("=" * 50)

df_features = prepare_features_for_country_year(df_img, method='flatten', limit=LIMIT_IMGS)
print(f"Размерность признаков: {df_features.shape}")

In [ ]:
df_features.head()

In [ ]:
print("\n" + "=" * 50)
print("ШАГ 3: Объединение с данными ВВП")
print("=" * 50)

df_merged = merge_with_gdp_data(df_features, df_gdp)

In [ ]:
df_merged.head()

#### 5.2. Datasets for training

In [ ]:
print("\n" + "=" * 50)
print("ШАГ 4: Разделение на выборки")
print("=" * 50)

train_df, val_df, test_df = split_data_by_country_and_year(df_merged)

### 6. CNN training

__CNN convolution as is__

![Image array](imgs/cnnkernel.png)

__CNN convolution kernels examples__

![Image array](imgs/cnnkernelexamples.png)

__CNN layers: why it works__

![Image array](imgs/cnncapture.png)

#### 6.1. Define CNN

Now correct our previous approach to utilize power of CNN:

```prompt
Теперь необходимо переделать модель предсказания GDP с использованием сверточных нейронных сетей
- модель будет построена на сверточных фильтрах
- не надо менять обработку и загрузку изображений, старайся использовать уже сделанный код
- используй образцы кода ниже, но измени задачу с классификации на регрессию

### Архитектура сети:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding='valid')
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding='valid')
        self.dropout1 = nn.Dropout(.5)
        self.dropout2 = nn.Dropout(.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)  # you may need to comment this line for HA
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)  # you may need to comment this line for HA
        x = self.fc2(x)
        output = F.softmax(x, dim=1)
        return output

model = SimpleCNN(NUM_CLASSES)
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=LEARNING_RATE
)
loss = nn.CrossEntropyLoss()

### Обучение сети:

for epoch in range(NUM_EPOCHS):
    correct = 0
    for i, (images, labels) in enumerate(train_loader):
        images = torch.autograd.Variable(images)
        labels = torch.autograd.Variable(labels)

        # Nullify gradients w.r.t. parameters
        optimizer.zero_grad()
        # forward propagation
        output = model(images)
        # compute loss based on obtained value and actual label
        compute_loss = loss(output, labels)
        # backward propagation
        compute_loss.backward()
        # update the parameters
        optimizer.step()

        # total correct predictions
        predicted = torch.max(output.data, 1)[1]
        correct += (predicted == labels).sum()
        if i % 50 == 0:
            print(
                'Epoch {} - training [{}/{} ({:.0f}%)] loss: {:.3f}, accuracy: {:.2f}%'.format(
                    epoch,
                    i * len(images),
                    len(train_loader.dataset),
                    100 * i / len(train_loader),
                    compute_loss.item(),
                    float(correct * 100) / float(BATCH_SIZE * (i + 1))
                ),
                end='\r'
            )

    # check total accuracy of predicted value and actual label
    accurate = 0
    total = 0
    for images, labels in test_loader:
        images = torch.autograd.Variable(images)
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        compute_loss = loss(output, labels)
        # total labels
        total += labels.size(0)

        # total correct predictions
        accurate += (predicted == labels).sum()
        accuracy_score = 100 * accurate/total

    print('Epoch {} - validation loss: {:.3f}, validation accuracy: {:.2f}%        '.format(
        epoch,
        compute_loss.item(),
        accuracy_score
    ))
```

In [ ]:
class GDPImageDataset(Dataset):
    """Датасет для изображений и целевых значений GDP."""
    
    def __init__(self, df: pd.DataFrame, feature_cols: List[str], target_col: str = 'gdp'):
        """
        Args:
            df: DataFrame с данными
            feature_cols: Список колонок с признаками (не используется, оставлено для совместимости)
            target_col: Название колонки с целевой переменной
        """
        self.df = df.reset_index(drop=True)
        self.target_col = target_col
        
        # Нормализуем GDP для стабильного обучения
        self.gdp_scaler = StandardScaler()
        gdp_values = self.df[target_col].values.reshape(-1, 1)
        self.gdp_normalized = self.gdp_scaler.fit_transform(gdp_values).flatten()
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        # Загружаем изображение (предполагаем, что оно уже обработано и сохранено)
        # В реальности здесь должна быть загрузка из файла
        # Но для экономии памяти предполагаем, что признаки уже извлечены
        # и хранятся в feature_cols как плоский вектор
        
        # Для CNN нам нужно восстановить 2D структуру изображения
        # Находим все колонки с признаками
        feature_cols = [col for col in self.df.columns if col.startswith('feature_')]
        features = row[feature_cols].values.astype(np.float32)
        
        # Восстанавливаем изображение из плоского вектора
        # Предполагаем, что изображение было 224x224 grayscale
        img_size = int(np.sqrt(len(features)))
        if img_size == TARGET_SIZE[0]:
            image = features.reshape(1, TARGET_SIZE[0], TARGET_SIZE[1])
        else:
            # Если размер не совпадает, создаем заглушку
            print(f"Warning: Unexpected feature size: {len(features)}")
            image = np.zeros((1, TARGET_SIZE[0], TARGET_SIZE[1]), dtype=np.float32)
        
        # Получаем нормализованное значение GDP
        gdp = self.gdp_normalized[idx].astype(np.float32)
        
        # Конвертируем в тензоры
        image_tensor = torch.from_numpy(image)
        gdp_tensor = torch.from_numpy(np.array([gdp]))
        
        return image_tensor, gdp_tensor


class SimpleCNNRegressor(nn.Module):
    """Простая CNN для регрессии GDP."""
    
    def __init__(self):
        super(SimpleCNNRegressor, self).__init__()
        
        # Сверточные слои
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        # Пуллинг и дропаут
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.3)

        if TARGET_SIZE[0] == 224:
            # Полносвязные слои
            # После 3 пуллингов: 224 -> 112 -> 56 -> 28
            self.fc1 = nn.Linear(128 * 28 * 28, 512)
            self.fc2 = nn.Linear(512, 128)
            self.fc3 = nn.Linear(128, 1)  # Один выход для регрессии
        elif TARGET_SIZE[0] == 128:
            # Полносвязные слои
            # После 3 пуллингов: 128 -> 64 -> 32 -> 16
            # Размер после сверток и пуллинга: 16x16
            # Количество каналов: 128
            # Размер для первого линейного слоя: 128 * 16 * 16 = 32768
            self.fc1 = nn.Linear(128 * 16 * 16, 512)
            self.fc2 = nn.Linear(512, 128)
            self.fc3 = nn.Linear(128, 1)  # Один выход для регрессии
        else:
            raise AttributeError("Target size should be 128 or 224")
        
    def forward(self, x):
        # Блок 1
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.pool(x)
        
        # Блок 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.pool(x)
        
        # Блок 3
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.pool(x)
        
        # Дропаут
        x = self.dropout(x)
        
        # Flatten
        x = torch.flatten(x, 1)
        
        # Полносвязные слои
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # Линейная активация для регрессии
        
        return x


def create_data_loaders(
    train_df: pd.DataFrame,
    val_df: pd.DataFrame,
    test_df: pd.DataFrame,
    feature_cols: List[str],
    batch_size: int = BATCH_SIZE
) -> Tuple[DataLoader, DataLoader, DataLoader, StandardScaler]:
    """Создает DataLoader'ы для обучения CNN.
    
    Args:
        train_df: Обучающая выборка
        val_df: Валидационная выборка
        test_df: Тестовая выборка
        feature_cols: Список колонок с признаками
        batch_size: Размер батча
        
    Returns:
        Кортеж (train_loader, val_loader, test_loader, gdp_scaler)
    """
    
    # Создаем датасеты
    train_dataset = GDPImageDataset(train_df, feature_cols)
    val_dataset = GDPImageDataset(val_df, feature_cols)
    test_dataset = GDPImageDataset(test_df, feature_cols)
    
    # Создаем DataLoader'ы
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2,
        pin_memory=True
    )
    
    return train_loader, val_loader, test_loader, train_dataset.gdp_scaler


def train_cnn_model(
    train_loader: DataLoader,
    val_loader: DataLoader,
    test_loader: DataLoader,
    gdp_scaler: StandardScaler,
    num_epochs: int = NUM_EPOCHS,
    learning_rate: float = LEARNING_RATE,
    device: str = None
) -> Tuple[nn.Module, Dict[str, List[float]]]:
    """Обучает CNN модель для регрессии GDP.
    
    Args:
        train_loader: DataLoader для обучающей выборки
        val_loader: DataLoader для валидационной выборки
        test_loader: DataLoader для тестовой выборки
        gdp_scaler: Scaler для обратной нормализации GDP
        num_epochs: Количество эпох
        learning_rate: Скорость обучения
        device: Устройство для обучения (cuda/cpu)
        
    Returns:
        Кортеж (обученная модель, история обучения)
    """
    
    # Определяем устройство
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Используется устройство: {device}")
    
    # Создаем модель
    model = SimpleCNNRegressor().to(device)
    
    # Оптимизатор и функция потерь (MSE для регрессии)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    
    # История обучения
    history = {
        'train_loss': [],
        'val_loss': [],
        'val_mape': []
    }
    
    # Обучение
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_batches = 0
        
        for i, (images, gdp_norm) in enumerate(train_loader):
            # Перемещаем на устройство
            images = images.to(device)
            gdp_norm = gdp_norm.to(device)
            
            # Forward pass
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, gdp_norm)
            
            # Backward pass
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            train_batches += 1
            
            # Вывод прогресса
            if i % 10 == 0:
                print(
                    f'Epoch {epoch} - training [{i * len(images)}/{len(train_loader.dataset)} '
                    f'({100. * i / len(train_loader):.0f}%)] loss: {loss.item():.4f}',
                    end='\r'
                )
        
        avg_train_loss = train_loss / train_batches
        history['train_loss'].append(avg_train_loss)
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_batches = 0
        val_predictions = []
        val_actuals = []
        
        with torch.no_grad():
            for images, gdp_norm in val_loader:
                images = images.to(device)
                gdp_norm = gdp_norm.to(device)
                
                outputs = model(images)
                loss = criterion(outputs, gdp_norm)
                
                val_loss += loss.item()
                val_batches += 1
                
                # Сохраняем для вычисления MAPE
                val_predictions.extend(outputs.cpu().numpy())
                val_actuals.extend(gdp_norm.cpu().numpy())
        
        avg_val_loss = val_loss / val_batches
        history['val_loss'].append(avg_val_loss)
        
        # Вычисляем MAPE (обратная нормализация GDP)
        val_predictions = np.array(val_predictions).reshape(-1, 1)
        val_actuals = np.array(val_actuals).reshape(-1, 1)
        
        val_predictions_orig = gdp_scaler.inverse_transform(val_predictions)
        val_actuals_orig = gdp_scaler.inverse_transform(val_actuals)
        
        # Избегаем деления на ноль
        mask = val_actuals_orig.flatten() != 0
        if mask.any():
            mape = np.mean(np.abs(
                (val_actuals_orig.flatten()[mask] - val_predictions_orig.flatten()[mask]) / 
                val_actuals_orig.flatten()[mask]
            )) * 100
        else:
            mape = np.inf
        
        history['val_mape'].append(mape)
        
        print(f'\nEpoch {epoch} - train loss: {avg_train_loss:.4f}, '
              f'val loss: {avg_val_loss:.4f}, val MAPE: {mape:.2f}%')
        
        # Ранняя остановка если MAPE не улучшается
        if epoch > 5 and mape > min(history['val_mape'][:-5]) * 1.1:
            print(f"Ранняя остановка на эпохе {epoch}")
            break
    
    # Финальная оценка на тестовой выборке
    model.eval()
    test_predictions = []
    test_actuals = []
    
    with torch.no_grad():
        for images, gdp_norm in test_loader:
            images = images.to(device)
            gdp_norm = gdp_norm.to(device)
            
            outputs = model(images)
            test_predictions.extend(outputs.cpu().numpy())
            test_actuals.extend(gdp_norm.cpu().numpy())
    
    # Обратная нормализация
    test_predictions = np.array(test_predictions).reshape(-1, 1)
    test_actuals = np.array(test_actuals).reshape(-1, 1)
    
    test_predictions_orig = gdp_scaler.inverse_transform(test_predictions)
    test_actuals_orig = gdp_scaler.inverse_transform(test_actuals)
    
    # Вычисляем метрики
    test_rmse = np.sqrt(mean_squared_error(test_actuals_orig, test_predictions_orig))
    test_r2 = r2_score(test_actuals_orig, test_predictions_orig)
    
    mask = test_actuals_orig.flatten() != 0
    if mask.any():
        test_mape = np.mean(np.abs(
            (test_actuals_orig.flatten()[mask] - test_predictions_orig.flatten()[mask]) / 
            test_actuals_orig.flatten()[mask]
        )) * 100
    else:
        test_mape = np.inf
    
    print('\n' + '='*50)
    print('ТЕСТОВЫЕ МЕТРИКИ')
    print('='*50)
    print(f'Test RMSE: {test_rmse:.2f}')
    print(f'Test R2: {test_r2:.4f}')
    print(f'Test MAPE: {test_mape:.2f}%')
    print('='*50)
    
    # Сохраняем тестовые метрики в историю
    history['test_rmse'] = test_rmse
    history['test_r2'] = test_r2
    history['test_mape'] = test_mape
    
    return model, history


def train_cnn_pipeline(
    train_df: pd.DataFrame,
    val_df: pd.DataFrame,
    test_df: pd.DataFrame,
    feature_cols: List[str]
) -> Tuple[nn.Module, Dict]:
    """Полный пайплайн обучения CNN для регрессии GDP.
    
    Args:
        train_df: Обучающая выборка
        val_df: Валидационная выборка
        test_df: Тестовая выборка
        feature_cols: Список колонок с признаками
        
    Returns:
        Кортеж (обученная модель, история обучения)
    """
    
    print("Создание DataLoader'ов...")
    train_loader, val_loader, test_loader, gdp_scaler = create_data_loaders(
        train_df, val_df, test_df, feature_cols
    )
    
    print("Обучение CNN модели...")
    model, history = train_cnn_model(
        train_loader, val_loader, test_loader, gdp_scaler
    )
    
    return model, history

#### 6.2. Training and results

In [ ]:
feature_cols = [col for col in df_merged.columns if col.startswith('feature_')]
cnn_model, history = train_cnn_pipeline(train_df, val_df, test_df, feature_cols)


In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.title('Training History')

plt.subplot(1, 2, 2)
plt.plot(history['val_mape'], label='Val MAPE')
plt.xlabel('Epoch')
plt.ylabel('MAPE (%)')
plt.legend()
plt.title('Validation MAPE')

plt.tight_layout()
plt.show()